<a href="https://colab.research.google.com/github/anukarri891/FMML_PROJECT_AND_LAB/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


1 **ANSWER**

Averaging validation accuracy across multiple splits can indeed provide more consistent and reliable results when evaluating the performance of a machine learning model. This technique is commonly referred to as "cross-validation," and it helps mitigate the impact of randomness in the data splitting process.

Here's how it works:

1. **Random Data Splitting**: When you train and validate a machine learning model, you typically split your dataset into two or more subsets, such as a training set and a validation set. The randomness in this splitting process can lead to variations in the performance metric (e.g., accuracy) obtained for different splits.

2. **K-Fold Cross-Validation**: One common approach is K-fold cross-validation, where you divide your dataset into K equally sized subsets (folds). You then train and validate your model K times, each time using a different fold as the validation set and the remaining folds as the training set. This process yields K different accuracy scores.

3. **Averaging Results**: After performing K-fold cross-validation, you calculate the average (and often the standard deviation) of the accuracy scores obtained in each iteration. This average accuracy score provides a more stable estimate of your model's performance on unseen data compared to a single validation split.

The benefits of averaging validation accuracy across multiple splits include:

- **Reduced Variance**: By using multiple splits, you reduce the influence of random variations in the data. A single random split may lead to overfitting or underestimating your model's true performance.

- **Better Generalization**: Averaging results over different subsets of data provides a more robust estimate of how well your model generalizes to unseen data. It helps you identify whether your model's performance is consistent or if it's just a result of a lucky/unlucky data split.

- **More Reliable Evaluation**: It helps in making more informed decisions about model hyperparameters, feature engineering, and model selection. It's particularly useful when dealing with smaller datasets.

However, it's important to note that cross-validation can be computationally expensive, especially when K is large, and it may not always be necessary or practical for very large datasets. In such cases, a single train-test split (e.g., an 80-20 or 70-30 split) may still provide a reasonable estimate of model performance.

In summary, averaging validation accuracy across multiple splits, typically using K-fold cross-validation, is a valuable technique for obtaining more consistent and reliable estimates of a model's performance and is particularly useful when working with smaller datasets or when you want to ensure robust generalization.

2 ANSWER

Averaging validation accuracy across multiple splits, such as through K-fold cross-validation, provides a more reliable and stable estimate of a model's performance on unseen data compared to a single validation split. However, it's important to note that this estimate is still an approximation and is intended to help you make more informed decisions about your model. It does not directly provide a more accurate estimate of the test accuracy.

Here's the distinction:

1. **Validation Accuracy**: The accuracy obtained during cross-validation or a single validation split is an estimate of how well your model is likely to perform on unseen data. It's a valuable tool for model selection, hyperparameter tuning, and assessing the model's generalization performance. However, it's still an estimate based on the validation data.

2. **Test Accuracy**: Test accuracy is the actual performance of your model on a completely separate and previously unseen dataset. This dataset is not used during training or model evaluation (validation). Test accuracy is the most accurate representation of how well your model is expected to perform in real-world scenarios.

While cross-validation and averaging validation accuracy are crucial for model development and evaluation, they are not a direct replacement for test accuracy. Test accuracy is the final measure of your model's performance and is used to assess its readiness for deployment in production.

To summarize, cross-validation helps you obtain a more reliable estimate of how well your model generalizes to unseen data, but it doesn't provide a more accurate estimate of the actual test accuracy. To obtain a test accuracy estimate, you need to reserve a separate, untouched dataset specifically for this purpose.

3RD ANSWRER

The choice of the percentage to reserve for the validation set should strike a balance between having enough data for model training and having a sufficient amount for model evaluation. There is no one-size-fits-all answer to this question, as the ideal split percentage depends on several factors, including the size of your dataset, the complexity of your model, and your specific goals. However, some common guidelines can help you make an informed decision:

1. **70-30 or 80-20 Split**: In many cases, a 70-30 or 80-20 split between the training and validation/test sets is a reasonable starting point. This means you reserve 70% or 80% of your data for training and use the remaining 30% or 20% for validation and testing. These splits provide a good balance between training data size and evaluation data size.

2. **Cross-Validation**: If you have a small dataset, consider using K-fold cross-validation. This approach divides your data into K subsets (or folds) and iteratively uses each fold as a validation set while the rest serve as the training set. Cross-validation helps maximize the use of your data for both training and evaluation and can provide more robust performance estimates.

3. **Stratified Split**: If your dataset is imbalanced (i.e., some classes have significantly fewer samples than others), consider using a stratified split. Stratified sampling ensures that the distribution of classes in your training, validation, and test sets remains representative of the overall dataset.

4. **Adjust Based on Data Size**: The larger your dataset, the more data you can afford to allocate for validation and testing. If you have a massive dataset, you might use a smaller percentage (e.g., 90% for training, 5% for validation, and 5% for testing) and still have ample data for model evaluation.

5. **Consider Your Goals**: Your specific goals and constraints also play a role. If you are primarily interested in rapid prototyping and model development, you might allocate more data to training. On the other hand, if you need highly accurate performance estimates, you might allocate a larger portion for validation and testing.

Ultimately, the choice of the validation/test split percentage is a trade-off. More data for training can help your model learn better, while more data for validation and testing can provide more accurate performance estimates. Experiment with different split percentages and evaluate how well your model generalizes to make an informed decision based on your specific circumstances. Cross-validation can also be a valuable tool for model evaluation when the dataset is limited.